In [1]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os

load_dotenv()

llm = ChatGroq(groq_api_key=os.environ['GROQ_API_KEY'], model="llama-3.3-70b-versatile")
response = llm.invoke("2+2")
response.content

'2 + 2 = 4'

In [15]:
from langchain_community.document_loaders import WebBaseLoader

# loader = WebBaseLoader("https://jobs.nike.com/job/R-33460")
loader = WebBaseLoader("https://jobs.apple.com/en-in/details/200602885/aiml-data-operations-annotation-analyst-flemish-12-month-contract?team=SFTWR")
page_data = loader.load().pop().page_content
print(page_data)




AIML Data Operations Annotation Analyst - Flemish - 12 Month. Contract - Jobs at Apple (IN) 













AppleStoreMaciPadiPhoneWatchAirPodsTV & HomeEntertainmentAccessoriesSupport


0+


    Please enable Javascript in your browser for best experience.
  

Jobs at AppleOpen MenuClose MenuWork at AppleLife at AppleProfileSign In
Search
AIML Data Operations Annotation Analyst - Flemish - 12 Month. ContractCork, County Cork, IrelandSoftware and ServicesAdd to Favourites AIML Data Operations Annotation Analyst - Flemish - 12 Month. ContractRemoved from favouritesSubmit CVBack to search resultsSummaryPosted: 
                May 02, 2025Weekly Hours: 39Role Number:200602885Imagine what you could do here. At Apple, new ideas have a way of becoming extraordinary products, services, and customer experiences very quickly. The people here at Apple donʼt just craft products - they build the kind of wonder that revolutionized entire industries. Itʼs the diversity of those people and their ide

In [16]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
print(res.content)
type(res.content)

```json
{
  "role": "AIML Data Operations Annotation Analyst - Flemish",
  "experience": "No specific experience mentioned, but Bachelor's degree or equivalent job-related experience preferred",
  "skills": [
    "Flemish reading, writing, and comprehension skills",
    "English reading, writing, and comprehension skills",
    "Active listening skills",
    "Understanding of local region and cultural variations",
    "Integrity and focus on customer privacy",
    "Attention to detail and critical thinking skills",
    "Organizational skills",
    "Ability to work independently and navigate systems and tools"
  ],
  "description": "We are looking for a motivated Annotation Analyst to join our Data Operations team. The successful candidate will annotate data, evaluate responses and information against guidelines, and use their language and cultural knowledge to support the customer experience. This is a 12-month contract position based in Cork, Ireland."
}
```


str

In [17]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'AIML Data Operations Annotation Analyst - Flemish',
 'experience': "No specific experience mentioned, but Bachelor's degree or equivalent job-related experience preferred",
 'skills': ['Flemish reading, writing, and comprehension skills',
  'English reading, writing, and comprehension skills',
  'Active listening skills',
  'Understanding of local region and cultural variations',
  'Integrity and focus on customer privacy',
  'Attention to detail and critical thinking skills',
  'Organizational skills',
  'Ability to work independently and navigate systems and tools'],
 'description': 'We are looking for a motivated Annotation Analyst to join our Data Operations team. The successful candidate will annotate data, evaluate responses and information against guidelines, and use their language and cultural knowledge to support the customer experience. This is a 12-month contract position based in Cork, Ireland.'}

In [18]:
import pandas as pd

df = pd.read_csv('my_portfolio.csv')
df


,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [19]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [ ]:
links = collection.query(query_texts=['Experience in python','Expertise in React Native'], n_results=2).get('metadatas')
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-native-portfolio'},
  {'links': 'https://example.com/react-portfolio'}]]

In [23]:
job = json_res
job['skills']

['Flemish reading, writing, and comprehension skills',
 'English reading, writing, and comprehension skills',
 'Active listening skills',
 'Understanding of local region and cultural variations',
 'Integrity and focus on customer privacy',
 'Attention to detail and critical thinking skills',
 'Organizational skills',
 'Ability to work independently and navigate systems and tools']

In [24]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas')
links

[[{'links': 'https://example.com/python-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/ios-ar-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/android-portfolio'},
  {'links': 'https://example.com/ios-portfolio'}],
 [{'links': 'https://example.com/ios-portfolio'},
  {'links': 'https://example.com/full-stack-js-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/ios-ar-portfolio'},
  {'links': 'https://example.com/android-portfolio'}]]

In [25]:
job

{'role': 'AIML Data Operations Annotation Analyst - Flemish',
 'experience': "No specific experience mentioned, but Bachelor's degree or equivalent job-related experience preferred",
 'skills': ['Flemish reading, writing, and comprehension skills',
  'English reading, writing, and comprehension skills',
  'Active listening skills',
  'Understanding of local region and cultural variations',
  'Integrity and focus on customer privacy',
  'Attention to detail and critical thinking skills',
  'Organizational skills',
  'Ability to work independently and navigate systems and tools'],
 'description': 'We are looking for a motivated Annotation Analyst to join our Data Operations team. The successful candidate will annotate data, evaluate responses and information against guidelines, and use their language and cultural knowledge to support the customer experience. This is a 12-month contract position based in Cork, Ireland.'}

In [29]:
prompt_email = PromptTemplate.from_template(
    """
    ### JOB DESCRIPTION:
    {job_description}
    
    ### INSTRUCTION:
    You are Mohan, a business development executive at TCS. TCS is a global leader in IT services, consulting, and business solutions, helping businesses around the world navigate their digital transformation journey. 
    Over the years, we have empowered numerous enterprises with innovative, cutting-edge technology solutions that drive operational efficiency, cost savings, and scalability.
    Your job is to write a cold email to the client regarding the job mentioned above describing the capability of TCS in fulfilling their needs.
    Also add the most relevant ones from the following links to showcase TCS's portfolio: {link_list}
    Remember you are Mohan, BDE at TCS.
    Do not provide a preamble.
    ### EMAIL (NO PREAMBLE): 
    """
)

chain_email = prompt_email | llm 
res = chain_email.invoke({"job_description": str(job), "link_list": links}) 
print(res.content)


Subject: Expert Annotation Analyst Services for Enhanced Customer Experience

Dear Hiring Manager,

I came across the job posting for an AIML Data Operations Annotation Analyst - Flemish, and I am excited to introduce TCS, a global leader in IT services, consulting, and business solutions, as a potential partner to fulfill your requirements. With our extensive experience in driving digital transformation for businesses worldwide, I am confident that we can provide the expertise you need to annotate data, evaluate responses, and support the customer experience.

Our team at TCS possesses the required Flemish and English language skills, active listening skills, and understanding of local region and cultural variations to deliver high-quality services. We prioritize integrity and focus on customer privacy, ensuring that our services meet the highest standards of data protection and confidentiality. Our experts are detail-oriented, critically thinking, and organized, with the ability to w